## Requirment Libraries

In [30]:

from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np

## Selenium Web Driver Setup

In [31]:
service = Service('C:/Users/fb_19/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')  # starts the browser in maximized window

options.add_argument('--disable-extensions')  # disables browser extensions
options.add_argument('--disable-gpu')  # disables hardware acceleration
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)


## Definition Of Element Lists

In [32]:

product_links = []
product_names = []
reviews = []
ids=[]
product_ratings=[]

## Reforming a Dataframe

In [33]:
df = pd.DataFrame(columns=("product-id","product-name","product-links","product-reviews","product-rating"))


## Collecting Product ID numbers

In [34]:
websites=("https://www.n11.com/bilgisayar/cevre-birimleri/mouse?pg=","https://www.n11.com/mutfak-gerecleri?pg=","https://www.n11.com/telefon-ve-aksesuarlari?pg=")
for a in websites:
    for i in range(1, 5):
        try:
            website= a
            website += str(i)
            driver.get(website)
            website = a
            names = driver.find_elements(By.CLASS_NAME, 'productName')
            
            for name in names:
                product_names.append(name.text)
            # Wait for the columns to load
            wait = WebDriverWait(driver, 10)
            columns = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'column')))

            for column in columns:


                column_contents = column.find_elements(By.CLASS_NAME, 'columnContent')


                for content in column_contents:


                    id_element = content.find_element(By.TAG_NAME, "a")
                    id = id_element.get_attribute("data-id")
                    product_link = id_element.get_attribute("href")
                    if id is not None:

                        ids.append(id)
                        product_links.append(product_link)
                    else:
                        continue


        except TimeoutException:
            print("TimeoutException error ignored.")
            continue

# Collecting Reviews

In [35]:
for a in ids:
    i=1
    website = "https://www.n11.com/component/render/groupProductReviews?page="+str(i)+"&productId="+str(a)
    driver.get(website)
    time.sleep(0.1)
    try:
        try:
            page= int(driver.find_element(By.XPATH, '//*[@id="reviewContent"]/div/div/span').text)
        except NoSuchElementException:
            page=1
        for i in range (1,page+1):
            website = "https://www.n11.com/component/render/groupProductReviews?page="+str(i)+"&productId="+str(a)
            driver.get(website)
            step1 = driver.find_elements(By.CLASS_NAME,'comment')
            for step in step1:
                try:
                    comment = step.find_element(By.TAG_NAME,'p').text
                    reviews.append(comment)
                    rating = step.find_element(By.CLASS_NAME,'commentTop').find_element(By.CLASS_NAME,'ratingCont').find_element(By.TAG_NAME,'span').get_attribute("class")
                    rating = int(rating.split("rating r")[1])/20
                    product_ratings.append(rating)
                    df.loc[len(df)]=[a,product_names[ids.index(a)],product_links[ids.index(a)],reviews[-1],product_ratings[-1]]
                except IndexError:
                    continue
    except TimeoutException:
        continue


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.57)
Stacktrace:
	GetHandleVerifier [0x00DC8CD3+51395]
	(No symbol) [0x00D35F31]
	(No symbol) [0x00BEE13A]
	(No symbol) [0x00BD25F2]
	(No symbol) [0x00C49B2B]
	(No symbol) [0x00C59E66]
	(No symbol) [0x00C43C26]
	(No symbol) [0x00C1C629]
	(No symbol) [0x00C1D40D]
	GetHandleVerifier [0x01146493+3711107]
	GetHandleVerifier [0x0118587A+3970154]
	GetHandleVerifier [0x01180B68+3950424]
	GetHandleVerifier [0x00E79CD9+776393]
	(No symbol) [0x00D41704]
	(No symbol) [0x00D3C5E8]
	(No symbol) [0x00D3C799]
	(No symbol) [0x00D2DDC0]
	BaseThreadInitThunk [0x75FF7BA9+25]
	RtlInitializeExceptionChain [0x777BBD2B+107]
	RtlClearBits [0x777BBCAF+191]


In [ ]:
len(reviews),len(product_ratings)

In [46]:
df.head(100000)

,product-id,product-name,product-links,product-reviews,product-rating
0,586873587,Logitech G300S Kablolu Optik Oyuncu Mouse,https://www.n11.com/urun/logitech-g300s-kablol...,Eksik geldi iki adet sipariş versim bir adet g...,5.0
1,586873587,Logitech G300S Kablolu Optik Oyuncu Mouse,https://www.n11.com/urun/logitech-g300s-kablol...,paketleme rezalet patpata bari sarın,1.0
2,586873587,Logitech G300S Kablolu Optik Oyuncu Mouse,https://www.n11.com/urun/logitech-g300s-kablol...,paketleme çok kötü paketi acarken video ve fot...,5.0
3,586873587,Logitech G300S Kablolu Optik Oyuncu Mouse,https://www.n11.com/urun/logitech-g300s-kablol...,güzel kullanışlı rahat ele oturuyor makro tuşl...,5.0
4,586873587,Logitech G300S Kablolu Optik Oyuncu Mouse,https://www.n11.com/urun/logitech-g300s-kablol...,Çok güzel ürün güzel kayıyor,5.0
...,...,...,...,...,...
9419,588616954,Narpump Şarjlı Tek Motorlu Damacana Pompası Beyaz,https://www.n11.com/urun/narpump-sarjli-tek-mo...,"Cihazın su tazyiği güzel, ucundaki metal borun...",3.0
9420,588616954,Narpump Şarjlı Tek Motorlu Damacana Pompası Beyaz,https://www.n11.com/urun/narpump-sarjli-tek-mo...,Fiyatına göre iş görür.,4.0
9421,588616954,Narpump Şarjlı Tek Motorlu Damacana Pompası Beyaz,https://www.n11.com/urun/narpump-sarjli-tek-mo...,ürünu bugun takıp kullandik ilk kullanım iyi z...,5.0
9422,588616954,Narpump Şarjlı Tek Motorlu Damacana Pompası Beyaz,https://www.n11.com/urun/narpump-sarjli-tek-mo...,ürün minik işi büyük tavsiye ederim,5.0


In [40]:
df.to_csv('data.csv')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9424 entries, 0 to 9423
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product-id       9424 non-null   object 
 1   product-name     9424 non-null   object 
 2   product-links    9424 non-null   object 
 3   product-reviews  9424 non-null   object 
 4   product-rating   9424 non-null   float64
dtypes: float64(1), object(4)
memory usage: 441.8+ KB


In [41]:
df["product-reviews"].duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9419    False
9420    False
9421    False
9422    False
9423    False
Name: product-reviews, Length: 9424, dtype: bool

In [45]:
df["product-reviews"][6627]

'teşekkürler'